In [1]:
import pandas as pd
import numpy as np
import os
import datetime as dt
from datetime import timedelta
from dateutil.relativedelta import *

In [2]:
seed = 7

In [3]:
import warnings
warnings.filterwarnings('ignore')

# Collect data

In [22]:
root_data='comodity-price-prediction-penyisihan-arkavidia-9'
hrg_bhn_pangan='Harga Bahan Pangan'
folder_train='train'
folder_mata_uang='Mata Uang'
folder_google_trend='Google Trend'

In [40]:
tanggal_awal=dt.datetime(year=2022,month=1,day=1)
tanggal_akhir=dt.datetime(year=2022,month=1,day=31)

In [30]:
list_bhn_pgn=[
    'Bawang Merah',         # bawang merah
    'Bawang Putih Bonggol', # bawang putih
    'Beras Medium',         # beras
    'Beras Premium',        # beras
    'Cabai Merah Keriting', # cabai merah
    'Cabai Rawit Merah',    # cabai rawit
    'Daging Ayam Ras',      # daging ayam
    'Daging Sapi Murni',    # daging sapi
    'Gula Konsumsi',        # gula
    'Minyak Goreng Curah',  # minyak goreng
    'Minyak Goreng Kemasan Sederhana',  #minyak goreng
    'Telur Ayam Ras',       # telur ayam
    'Tepung Terigu (Curah)',# tepung terigu
]

In [31]:
list_google_trend={
    'Bawang Merah': 'bawang merah',
    'Bawang Putih Bonggol': 'bawang putih',
    'Beras Medium':'beras',
    'Beras Premium':'beras',
    'Cabai Merah Keriting':'cabai merah',
    'Cabai Rawit Merah':'cabai rawit',
    'Daging Ayam Ras':'daging ayam',
    'Daging Sapi Murni':'daging sapi',
    'Gula Konsumsi':'gula',
    'Minyak Goreng Curah':'minyak goreng',
    'Minyak Goreng Kemasan Sederhana':'minyak goreng',
    'Telur Ayam Ras':'telur ayam',
    'Tepung Terigu (Curah)':'tepung terigu',
}

In [32]:
list_nama_daerah=[
    'Aceh',
    'Bali',
    'Banten',
    'Bengkulu',
    'DI Yogyakarta',
    'DKI Jakarta',
    'Gorontalo',
    'Jambi',
    'Jawa Barat',
    'Jawa Tengah',
    'Jawa Timur',
    'Kalimantan Barat',
    'Kalimantan Selatan',
    'Kalimantan Tengah',
    'Kalimantan Timur',
    'Kalimantan Utara',
    'Kepulauan Bangka Belitung',
    'Kepulauan Riau',
    'Lampung',
    'Maluku Utara',
    'Maluku',
    'Nusa Tenggara Barat',
    'Nusa Tenggara Timur',
    'Papua Barat',
    'Papua',
    'Riau',
    'Sulawesi Barat',
    'Sulawesi Selatan',
    'Sulawesi Tengah',
    'Sulawesi Tenggara',
    'Sulawesi Utara',
    'Sumatera Barat',
    'Sumatera Selatan',
    'Sumatera Utara'
]

In [33]:
path_data_source=os.path.join(root_data,hrg_bhn_pangan,folder_train)

In [34]:
data_bhn_pgn={
    'Tanggal':[],
    'Bahan_pangan':[],
    'Daerah':[],
    'Harga_lag_1':[],
    'Harga_lag_7':[],
    'Harga_lag_30':[],
    'Google_trends':[],
    'MYRUSD':[],
    'SGDUSD':[],
    'THBUSD':[],
    'USDIDR':[],
    'Harga':[]
}
df_bhn_pgn_all=pd.DataFrame(data_bhn_pgn)

In [45]:
myrusd_path=os.path.join(root_data,folder_mata_uang,'MYRUSD=X.csv')
df_myrusd=pd.read_csv(myrusd_path)
df_myrusd['Date']=pd.to_datetime(df_myrusd['Date'])
sgdusd_path=os.path.join(root_data,folder_mata_uang,'SGDUSD=X.csv')
df_sgdusd=pd.read_csv(sgdusd_path)
df_sgdusd['Date']=pd.to_datetime(df_sgdusd['Date'])
thbusd_path=os.path.join(root_data,folder_mata_uang,'THBUSD=X.csv')
df_thbusd=pd.read_csv(thbusd_path)
df_thbusd['Date']=pd.to_datetime(df_thbusd['Date'])
usdidr_path=os.path.join(root_data,folder_mata_uang,'USDIDR=X.csv')
df_usdidr=pd.read_csv(usdidr_path)
df_usdidr['Date']=pd.to_datetime(df_usdidr['Date'])

myrusd=0
sgdusd=0
thbusd=0
usdidr=0

In [46]:
for date in [tanggal_awal + dt.timedelta(days=x) for x in range((tanggal_akhir - tanggal_awal).days + 1)]:
    for nama_bhn_pgn in list_bhn_pgn:
        file_bhn_pgn=os.path.join(path_data_source, nama_bhn_pgn+'.csv')
        df_bhn_pgn=pd.read_csv(file_bhn_pgn)
        try:
            df_bhn_pgn['Date']=pd.to_datetime(df_bhn_pgn['Date'])
        except Exception as e:
            df_bhn_pgn['Tanggal']=pd.to_datetime(df_bhn_pgn['Tanggal'])
        yesterday = date - timedelta(days=1)
        week_ago = date - timedelta(days=7)
        month_ago = date - relativedelta(months=1)
        for daerah in list_nama_daerah:
            try:
                gt_path=os.path.join(root_data,folder_google_trend,list_google_trend[nama_bhn_pgn],daerah+'.csv')
                df_gt=pd.read_csv(gt_path)
                df_gt['Date']=pd.to_datetime(df_gt['Date'])
            except Exception as e:
                df_gt=None
            
            if not df_myrusd.query('Date == @date').empty and not df_usdidr.query('Date == @date').empty:
                myrusd=round(df_myrusd.query('Date == @date')['Adj Close'].values[0]*df_usdidr.query('Date == @date')['Adj Close'].values[0], 4) if not df_myrusd.query('Date == @date').empty else np.int32(0)
            if not df_sgdusd.query('Date == @date').empty and not df_usdidr.query('Date == @date').empty:
                sgdusd=round(df_sgdusd.query('Date == @date')['Adj Close'].values[0]*df_usdidr.query('Date == @date')['Adj Close'].values[0], 4) if not df_sgdusd.query('Date == @date').empty else np.int32(0)
            if not df_thbusd.query('Date == @date').empty and not df_usdidr.query('Date == @date').empty:
                thbusd=round(df_thbusd.query('Date == @date')['Adj Close'].values[0]*df_usdidr.query('Date == @date')['Adj Close'].values[0], 4) if not df_thbusd.query('Date == @date').empty else np.int32(0)
            if not df_usdidr.query('Date == @date').empty:
                usdidr=round(df_usdidr.query('Date == @date')['Adj Close'].values[0], 4) if not df_myrusd.query('Date == @date').empty else np.int32(0)
            new_data={
                'Tanggal':date,
                'Bahan_pangan':nama_bhn_pgn,
                'Daerah':daerah,
                'Harga_lag_1':np.int32(df_bhn_pgn.query('Date == @yesterday')[daerah].values[0]) if not df_bhn_pgn.query('Date == @yesterday').empty else np.int32(0),
                'Harga_lag_7':np.int32(df_bhn_pgn.query('Date == @week_ago')[daerah].values[0]) if not df_bhn_pgn.query('Date == @week_ago').empty else np.int32(0),
                'Harga_lag_30':np.int32(df_bhn_pgn.query('Date == @month_ago')[daerah].values[0]) if not df_bhn_pgn.query('Date == @month_ago').empty else np.int32(0),
                'Google_trends':df_gt.query('Date == @date')[list_google_trend[nama_bhn_pgn]].values[0] if df_gt is not None and not df_gt.query('Date == @date').empty else np.int32(0),
                'MYRUSD':myrusd,
                'SGDUSD':sgdusd,
                'THBUSD':thbusd,
                'USDIDR':usdidr,
                'Harga':np.int32(df_bhn_pgn.query('Date == @date')[daerah].values[0]) if not df_bhn_pgn.query('Date == @date').empty else np.int32(0)
            }
            df_bhn_pgn_all = df_bhn_pgn_all._append(new_data, ignore_index=True)

In [26]:
df_bhn_pgn_all.head(100)

,Tanggal,Bahan_pangan,Daerah,Harga_lag_1,Harga_lag_7,Harga_lag_30,Google_trends,MYRUSD,SGDUSD,THBUSD,USDIDR,Harga
0,2022-01-01,Bawang Merah,Aceh,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,28970.0
1,2022-01-01,Bawang Merah,Bali,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,20870.0
2,2022-01-01,Bawang Merah,Banten,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,26890.0
3,2022-01-01,Bawang Merah,Bengkulu,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,26650.0
4,2022-01-01,Bawang Merah,DI Yogyakarta,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,25240.0
...,...,...,...,...,...,...,...,...,...,...,...,...
95,2022-01-01,Beras Medium,Sulawesi Selatan,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,9550.0
96,2022-01-01,Beras Medium,Sulawesi Tengah,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,10070.0
97,2022-01-01,Beras Medium,Sulawesi Tenggara,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,10220.0
98,2022-01-01,Beras Medium,Sulawesi Utara,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,10940.0


In [47]:
df_bhn_pgn_all.to_csv('test.csv', index=False)

In [16]:
file_bhn_pgn=os.path.join(path_data_source, list_bhn_pgn[0]+'.csv')
df_bhn_pgn=pd.read_csv(file_bhn_pgn)
gt_path=os.path.join(root_data,folder_google_trend,list_google_trend[list_bhn_pgn[0]],list_nama_daerah[0]+'.csv')
print(f'gt_path: {gt_path}')
df_gt=pd.read_csv(gt_path)
df_gt['Date']=pd.to_datetime(df_gt['Date'])
date=dt.datetime(year=2022,month=7,day=26)
df_gt.query('Date == @date')['bawang merah'].values[0]

gt_path: comodity-price-prediction-penyisihan-arkavidia-9\Google Trend\bawang merah\Aceh.csv


np.float64(49.50915254237288)

In [51]:
list_bhn_pgn[0]

'Bawang Merah'

In [53]:
list_google_trend['Bawang Merah']

'bawang merah'

In [54]:
list_google_trend[list_bhn_pgn[0]]

'bawang merah'